<h1> Formalizzazione del problema </h1>

Quando c'e' un grande quantitativo di dati da analizzare, risulta indispensabile compiere un'analisi su quelli utilizzabili piuttosto che interessanti. Un modo per svolgere tale selezione e' comprendere quali sono gli aspetti del problema, in questione, importanti e/o necessari per risolverlo. Ovvero discriminare le parti rilevanti all'esperienza da quelle che non lo sono. Per fare cio' esistono due tecniche: Feature Selection e Feature Extraction.

<h1> Feature Selection </h1>

La Feature Selection viene praticata di continuo dalla mente umana,  perche' non fa altro che eliminare caratteristiche ridondanti  o con informazione poco utile, riducendo lo spazio di dimensionalita' delle features. Tale pratica e' utile quando si vuole migliorare l'interpretabilita' del modello.

<h1>Feature Extraction </h1>

La Feature Extraction svolge una riduzione della dimensionalita' delle features, perdendone pero' in interpretabilita' del modello. Questo perche' le features originali vengono combinate tra loro, per ottenere quelle del modello.

Una tecnica che implementa la Feature Extraction e' la PCA. Di seguito tratto nel dettaglio tale tecnica.

<h3> PCA </h3>
Un esempio di Feature Extraction e' la PCA (Principal Component Analysis), tecnica per la semplificazione dei dati utilizzata nel campo della statistica multivariata. Il suo scopo e' quello di ridurre il numero, piu' o meno elevato, di variabili, che descrivono un dataset, a un numero inferiore di variabili latenti, limitando al massimo la perdita d'informazione.

La PCA, algoritmo non supervisionato, puo' avere una duplice visione:
1. come algoritmo di riduzione della dimensionalita': svolge una trasformazione lineare delle variabili, proiettando quelle originali in un nuovo sistema cartesiano, in cui la nuova variabile con maggiore varianza viene proiettata sul primo asse, e la nuova variabile seconda per dimensione di varianza, nel secondo (cosi avviene la combinazione delle features) e cosi via; 
2. come strumento di visualizzazione, per il filtraggio del rumore, estrazione e ingegneria delle features, ecc.. 

Per studiare l'applicazione della PCA, di seguito:
- implemento un piccolo esempio, composto da un set di dati bidimensionale di 150 campioni;
- studio le conseguenze della PCA sul dataset `Breast Cancer Wisconsin (Diagnostic) Data Set.`

<h5>Applicazione della PCA su dataset di prova (150 esempi)</h5>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num = np.random.RandomState(1) # viene restituito un array 1-D riempito con i valori generati
X = np.dot(num.rand(2, 2), num.randn(2, 150)).T
# prodotto scalare tra una matrice 2x2 e una matrice 2x150
# T fa la trasposta

plt.rcParams['figure.figsize'] = [10, 5] # ridimensiono l'area di stampa del plot
plt.scatter(X[:, 0], X[:, 1], c = 'purple')
plt.axis('equal'); # imposto la scala degli assi uguale

Come appare dalla figura sopra e' evidente una relazione tra gli assi x e y. Difatti essendo, questo un problema di apprendimento senza supervisione, l'obiettivo e' quello di conoscere la relazione che esiste tra i due assi, e non come farebbe la regressione, prevedere i valori di y da x.\
Nella PCA, tale relazione viene quantificata individuando un elenco degli assi principali nei dati, e utilizzando tali assi per descrivere il set di dati in analisi.

La PCA, sul mio dataset d'esempio, la calcolo nel modo che segue:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)

Tale processo permette di apprendere alcune quantita' di dati. Tra le quali i "componenti" e la "varianza spiegata".


In [ ]:
print(pca)
print("componenti della pca: " + str(pca.components_)) # componenti
print("varianza spiegata della pca: " + str(pca.explained_variance_)) # varianza spiegata

Queste possono venire trasformate in un vettore, tale che:
- componenti: e' la direzione del vettore;
- varianza spiegata: e' la lunghezza del vettore.

E permettono di definire quali sono gli assi principali dei dati ("componenti") e quanto e' importante ogni asse principale, in rapporto alla distribuzione dei dati ("varianza spiegata").

Per generare suddetto vettore ho implementato il metodo `vector_build`.



In [ ]:
def vector_build(v,w,ax):
    ax = ax or plt.gca() # ottengo gli assi correnti
    arrowprops = dict(arrowstyle='->', linewidth=2)
    # s: parametro e' il testo dell'annotazione
    # xy: parametro e' il punto (x, y) da annotare
    # xytext: parametro facoltativo. È la posizione (x, y) in cui posizionare il testo
    # arrowprops: parametro opzionale e contiene il tipo dict. Il suo valore predefinito e' None
    ax.annotate('', w, v, arrowprops=arrowprops)

plt.scatter(X[:, 0], X[:, 1], alpha=0.4, color="purple")
# zip accoppia gli elementi, mediante iteratore, in ordine di comparizione
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 2 * np.sqrt(length)
    vector_build(pca.mean_, pca.mean_ + v, None)
plt.axis('equal');

L'uso della PCA, come ho gia' affermato, comporta la riduzione di dimensionalita'; cioe' l'azzerramento di uno o piu' componenti principali, risultando di una proiezione dimensionale inferiore, preservando la varianza massima dei dati. Tale trasformata appare evidente se svolgo, sul mio esempio di prova, le seguenti chiamate ai metodi propri della PCA, implementati da Scikit-Learn:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X) 	# adatta il modello con X
X_pca = pca.transform(X)  # applica la riduzione della dimensionalità a X
print("Dimensione originale: ", X.shape)
print("Dimensione dopo l'applicazione della pca: ",X_pca.shape)

Ecco che da un array a 2 dimensioni mi sono ricondotta a un array in una singola dimensione.

E' interessante da vedere l'effetto che ha tale riduzione sul dataset, per questo eseguo la trasformazione inversa dei dati ridotti (`X_pca_inv`) per tracciarla insieme ai dati originali (`X`).

In [ ]:
X_pca_inv = pca.inverse_transform(X_pca) # trasforma i dati nel loro spazio originale
plt.scatter(X[:,0], X[:,1], alpha=0.2, color="purple")
plt.scatter(X_pca_inv[:,0], X_pca_inv[:,1], alpha=0.8, color="purple")
plt.axis("auto")
plt.show()

In figura sopra, i punti piu' chiari sono i dati originali, invece quelli piu' scuri la loro versione proiettata. Questo mi permette di comprendere il significato di "riduzione di dimensionalita'" per la PCA: le informazioni lungo l'asse o gli assi principali meno importanti vengono rimosse, lasciando solo le componenti con varianza superiore. La frazione di varianza tagliata, proporzionale all'area dei punti attorno alla linea di distribuzione, rappresenta quanta informazione viene scartata nella riduzione di dimensionalita'.

In questo specifico set di dati sono passata da due a un'unica dimensione, cio' significa che ho avuto una riduzione di dimensionalita' del ben 50%; tuttavia rimangono evidenti quali sono le relazioni fondamentali tra i punti. Difatti la PCA, ha come punto di forza, preservare la relazione complessiva dei dati del dataset.

Applicazione della PCA su `Breast Cancer Wisconsin (Diagnostic) Data Set`

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        df = os.path.join(dirname, filename)
        print(df)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


**Caricamento del dataframe**

In [ ]:
breast_cancer_df =  pd.read_csv(df, usecols=[i for i in range(0, 32)],encoding='latin-1')  # costruisco il dataframe
breast_cancer_df.head(10) # stampo delle prime 10 righe del dataframe

**Creazione del dataset**

Salvo il dataset su una struttura dati array e rendo il campo `diagnosis`uniforme al resto dei dati: 0 se il il tumore e' benigno (B), altrimenti 1 se e' maligno (M).

In [ ]:
X = breast_cancer_df.values # data set
X[X == 'B'] = 0 # benigno
X[X == 'M'] = 1 # maligno


**Caratteristiche del dataset**

Nel dataset in analisi si hanno:
- numero di features: 32;
- numero di valori per ogni features: 569.

In [ ]:
print("Dimensione originale: " + str(X.shape))

E il contenuto e' quanto stampato di seguito.

In [ ]:
print(X)

**Standaridazzazione delle features**

Procedo a standardizzare le features. Per farlo ho deciso di usare il metodo di standardizzazione (`StanderScaler`).
La formula e':
<center> $\frac{X - \mu}{\sigma}$</center>
ove X rappresenta il vettore di dati da standardizzare, $\mu$ la media e $\sigma$  la devianza standard.

Per svolgere una standardizzazione conforme, procedo:
1. a calcolare $\mu$ e $\sigma$ sui dati di X;
2. a standardizzare X sulla base del calcolo svolto al punto (1).

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler() # richiamo la classe StandardScaler(), inizializzo un oggetto e lo assegno alla variabile sc
sc.fit(X) # stimo la media del campione e la deviazione standard di X
X_std = sc.fit_transform(X) # standardizzazione di X con quanto calolato da fit

**Applicazione della PCA**

In [ ]:
pca = PCA(n_components=2)
X_std_pca = pca.fit(X_std)
X_pca = pca.transform(X_std)  # applica la riduzione della dimensionalità a X

Con l'applicazione della PCA, come mi aspettavo, i nuovi dati (`X_std_pca`) sono stati ridotti da 32 a 2 features, invece il numero di righe sono rimaste immutate rispetto al dataset di dati originale (`X`).

In [ ]:
print("Dimensione originale: " + str(X.shape))
print("Dimensione dopo l'applicazione della pca: " + str(X_pca.shape))

Di seguito mostro alcuni risultati che ho ottenuto dall'applicazione della tecnica.

In [ ]:
print("####### PCA #######")
print(pca)
print("componenti: ")
print(pca.components_) # componenti
print("varianza spiegata: ")
print(pca.explained_variance_) # varianza spiegata
print("dataset ottenuto al termine dell'applicazione della tecnica: ")
print(X_pca)

Per confrontare la distribuzione, sulle prime due componeti principali del dataset originale con il dataset sottoposto a sandardizzazione e PCA, ho proceduto con la generazione di plot.

In [ ]:
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.scatter(X[:, 0], X[:, 1], alpha=0.5, color = "purple")
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5, color = "orange")
plt.legend(['valori originali','valori post-pca'], numpoints=1)
plt.axis('tight'); # imposto limiti abbastanza grandi da mostrare tutti i dati
plt.show()


Faccio seguire, per maggiore chiarezza della distribuzione, anche i due grafici distinti.

In [ ]:
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.scatter(X[:, 0], X[:, 1], alpha=0.5, color = "purple")
plt.axis('tight'); # imposto limiti abbastanza grandi da mostrare tutti i dati
plt.legend(['valori originali'], numpoints=1)
plt.show()

plt.xlabel('component 1')
plt.ylabel('component 2')
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5, color = "orange")
plt.axis('tight'); # imposto limiti abbastanza grandi da mostrare tutti i dati
plt.legend(['valori post-pca'], numpoints=1)
plt.show()

Dai grafici sopra e' evidente la riduzione di dimensionalita' a seguito della PCA. 
Difatti come si puo' vedere dai punti in giallo, l'informazione viene riassunta nei primi due assi evitando situazione come:
- pedita d'informazione;
- un'eccessiva distribuzione dei dati in uno spazio troppio ampio e percio' complesso da scandagliare. Come si puo' vedere, infatti i punti in viola non hanno una distribuzione omogenea gia' nelle prime due features, dunque per 30 implica un lavoro con una complessita' sia in tempo che in spazio considerevole;
-  la mancanza di somiglianza tra le features o un'eccessiva dispersione dei valori, puo' generare situazioni di cattiva interpretazione dei dati. Quest'ultimo aspetto e' particolarmente rilevante quando si devono svolgere stime, su dati, che non fanno uso di metodi di apprendimento, ma di calcoli di frequenza, e che percio' tendono a sottovalutare il valore di punti isolati.